<a href="https://colab.research.google.com/github/NataliaGaravito/sistema-motor-generador-aplicado-a-Hardware-In-the-Loop-con-Machine-Learning/blob/main/HIL_entrenamiento_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
import pandas as pd
import numpy as np
from numpy import  delete
import sklearn
from sklearn import  preprocessing
from sklearn.utils import shuffle
from sklearn.neural_network import MLPRegressor
from matplotlib import pyplot as plt
from sklearn.metrics import  mean_squared_error
from sklearn import linear_model

In [ ]:
xls=pd.ExcelFile('ppid.xls') #llamar archivo
#print(xls.sheet_names)   #mirar nombre de las hojas de calculo
datos=xls.parse('Hoja1')#back up de datos originales 
df=pd.DataFrame(datos)#para preprocesamiento de datos 
print (df)
#print(df.head(1500))

In [ ]:
max=pd.Series.max(df)
print(max)
min=pd.Series.min(df)
print(min)

In [ ]:
scaler = preprocessing.MinMaxScaler()
dfn = scaler.fit_transform(df)
df=pd.DataFrame(dfn)#convertir "tabla" otra vez
df.columns=['PWM1', 'PWM11', 'PWM12', 'PWM13', 'PWM14','PWM15', 'RES1', 'RES11', 'RES12', 'RES13', 'RES14', 'RES15', 'V1G', 'V1G1', 'V1G2', 'V1G3'] #asignar nombre 

In [ ]:
max=pd.Series.max(df)
print(max)
min=pd.Series.min(df)
print(min)

In [ ]:
#descargar archivos normalizados
df.to_excel('Libro2.xlsx',index=False)

## **eliminacion outliers**

grafica cajas y bigotes

In [ ]:
df.boxplot(return_type='dict') #grafica de cajas y bigotes (outliers)
plt.show()

metodo 3 sigma/ eliminacion de filas

In [ ]:
print(len(df))
for i in range(11):#numero de columnas
  dk=np.array(df)
  coms=dk[:,i]>((np.mean(dk[:,i]))+(3*(np.std(dk[:,i]))))#formula para sigma 3 valores superiores
  comi=dk[:,i]<((np.mean(dk[:,i]))-(3*(np.std(dk[:,i]))))#formula para sigma 3 valores inferiores
  df.drop(df.index[coms|comi], inplace=True)#eliminar valores fuera de sigma
print(df)

eliminacion de columnas

In [ ]:
#correlacion entre columnas
df.corr(method='pearson')

In [ ]:
#datos eliminados con poca correlacion
del df['PWM14']
del df['PWM15']
del df['RES14']
del df['RES15']
del df['V1G3']
del df['V1G2']

In [ ]:
#dividir datos de entradas y salidas 
X=np.array(df.drop(['V1G'],axis=1))#entradas/se eliminan las salidas 
Y=np.array(df.drop(['PWM1', 'PWM11', 'PWM12', 'PWM13', 'RES1', 'RES11', 'RES12', 'RES13', 'V1G1'],axis=1))#salidas/se eliminan las entradas

print (len(X))
print (len(Y))

#dividir datos de entrenamiento y de prueba
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(X, Y, test_size=0.3)

#entrenamiento de la red neuronal
rgr=MLPRegressor(solver='lbfgs', alpha=1,activation='logistic', 
                 hidden_layer_sizes=(4), 
                 learning_rate='invscaling', max_iter=300)     
#invscaling          adaptive        random_state=1,

rgr.fit(x_train, y_train) 
rgr.get_params()

In [ ]:
mp=[coef.shape for coef in rgr.coefs_] #matrices de peso de los parametros del modelo
res=rgr.predict(x_test) #resultado de la prediccion
sco=rgr.score(x_test,y_test) # puntuacion de exactitud del resultado de la prediccion
error=mean_squared_error(res, y_test, squared=False)#MSE=>squared=True / RMSE=>squares=False

peso=rgr.coefs_ #matriz peso de las neuronas
inpeso=rgr.intercepts_ #vector peso de neurona vacia

print ('matrices ', mp,'\n')
#print('resultado prediccion ', res,'\n')
print('coeficiente prediccion ', sco,'\n')
print('peso coeficientes ', peso,'\n')
print('peso neurona vacia ', inpeso,'\n')
print('error', error,'\n')

In [ ]:
#grafica de aprendizaje (x= # iteraciones, y= error)
plt.figure()
plt.subplot(211)
plt.plot(y_test, color='orange')
plt.xlim(5000,5050)
plt.plot(res, color='blue')